In [1]:
import waybackpy as wbp
import savepagenow as spn
import pandas as pd
import mariadb
import getpass
import mysql.connector

In [2]:
pw = getpass.getpass()
con = mysql.connector.connect(
# mariadb.connect(
    user = "scraper",
    password = pw,
    host = "127.0.0.1",
    port = 3336,
    database = "russian_media_database_rt"
)
cursor = con.cursor()

In [6]:
# get domains:
versions = pd.read_sql_query("SELECT * FROM sitemap_versions", con)
versions = versions.link.str.slice(0,-12)

/tmp/ipykernel_7997/3485014270.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  versions = pd.read_sql_query("SELECT * FROM sitemap_versions", con)


In [13]:
done = pd.read_sql_query("SELECT link FROM archived_pages", con)
len(done)

/tmp/ipykernel_7997/1435864534.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  done = pd.read_sql_query("SELECT link FROM archived_pages", con)


4029050

In [12]:
# get all archived urls: 

from sqlalchemy import create_engine


username = "scraper"
host = "127.0.0.1"
port = "3336"
database = "russian_media_database_rt"

# Create a connection string
connection_str = f"mysql+mysqlconnector://{username}:{pw}@{host}:{port}/{database}"

# Create an SQLAlchemy engine
engine = create_engine(connection_str)



In [8]:
versions = versions[4]

In [11]:
versions = [versions, "https://lat.rt.rs"]
versions

['https://russian.rt.com', 'https://lat.rt.rs']

In [14]:

for base_url in versions:

    with engine.begin() as connection:

        archive_urls = []

        wayback_base = wbp.Url(base_url)
        archive = wayback_base.known_urls(subdomain=True)

        for url in archive:
            # print(url)
            # write to database:
            archive_urls.append(url)

        archive_urls = [url for url in archive_urls if url not in done]

        archive_urls_df = pd.DataFrame(archive_urls, columns=['link'])

        archive_urls_df.to_sql('archived_pages', con = connection, if_exists = 'append', index=False , chunksize = 1000)

        connection.commit()
    #    for url in archive_urls:
    #        cursor.execute("INSERT INTO archived_pages (link) VALUES (%s)", (url,))
    #        con.commit()    

        print("saved",  base_url, len(archive_urls))Milan Schröder

saved https://russian.rt.com 2215828
saved https://lat.rt.rs 14732


In [ ]:
archive_urls = pd.read_sql_query("SELECT * FROM archived_pages;", con)
len(archive_urls)

284506

In [ ]:
# create df:
url_df = pd.DataFrame({'link' = archive_urls, 'archived': 1})  

In [ ]:
# compare to self-scraped pages: 

url_list = pd.read_sql_query("SELECT link, doc_hash, available_online FROM page_data;", con)
url_list

/tmp/ipykernel_321956/660494829.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  url_list = pd.read_sql_query("SELECT link, doc_hash, available_online FROM page_data limit 10;", con)


,link,doc_hash,available_online
0,https://rt.rs/tag/APEK/,None,0
1,https://rt.rs/tag/AS-Monako/,None,0
2,https://rt.rs/tag/ATP-Tenis/,None,0
3,https://rt.rs/tag/Aba-liga-kosarka/,None,0
4,https://rt.rs/tag/Afrika/,None,0
5,https://rt.rs/tag/Ajaks/,None,0
6,https://rt.rs/tag/Ajatolah-Ali-Hamnej/,None,0
7,https://rt.rs/tag/Ajfon/,None,0
8,https://rt.rs/tag/Ajntraht-Frankfurt/,None,0
9,https://rt.rs/tag/Alba-Berlin/,None,0


In [ ]:
# join archived and scraped pages:
# how many of the scarped pages are archived?
# how many of the archived pages are scraped?

compare = pd.read_sql_query("""
    SELECT * link, available_online, doc_hash
    FROM page_data
    FULL JOIN archived_pages
    ON page_data.link = archived_pages.link
    ;
""", con)

In [ ]:
# write to database:
for i in range(len(archive_urls)):
    cursor.execute("INSERT INTO archived_pages (link) VALUES ('{archive_urls[i]}')")

In [ ]:
# loop over df:
for index, row in url_df.iterrows():
    cursor.execute("INSERT INTO archived_pages (link, archived) VALUES (?, ?)", row.link, row.archived)

In [13]:
# create table
sql_command = """CREATE TABLE IF NOT EXISTS archived_pages (
	rowid BIGINT(20) NOT NULL AUTO_INCREMENT,
    link TEXT,
	PRIMARY KEY (`rowid`) USING BTREE
    );
"""

# execute the statement
cursor.execute(sql_command)

In [ ]:
# commit changes:
con.commit()


In [ ]:
# clsoe connection:
con.close()